How's it look?

Now with **Mixture model**!

# Make a pandas dataframe of the average values from the inference

In [1]:
import pandas as pd

In [2]:
old_df = pd.read_csv('../data/analysis/IGRINS_LkCa4_run01_last20kMCMC.csv')

Copy the metadata columns from the old dataframe:

In [3]:
new_df = old_df[old_df.columns[np.arange(7)]]

What columns do we want?

- All the same as the last array: median and 2 sigma ranges. (6+6)*3 = 36 
- Teff2 and Omega2 and 2 sigma ranges. (1+1)*3 = 6
- Conversion of Omega2 and Omega1 into fill factor and 2 sigma ranges. (1)*3 = 3
- Metadata about the number of samples and the burn-in assumed to compute the above values.
    - We can get more metadata from the `sacct` log from slurm

In [4]:
for col in old_df.columns[7:].values:
    new_df[col] = np.NaN

//anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [5]:
for col in ['Teff2_05p', 'Teff2_50p', 'Teff2_95p']:
    new_df[col] = np.NaN
for col in ['logO2_05p', 'logO2_50p', 'logO2_95p']:
    new_df[col] = np.NaN
for col in ['ff_05p', 'ff_50p', 'ff_95p']:
    new_df[col] = np.NaN

In [6]:
new_df['inference'] = np.NaN

### $K-$band

In [9]:
ms = np.arange(72, 94+1)

In [10]:
for m in ms:
    ind = new_df.index[new_df.m_val == m]
    try:
        ws = np.load("../sf/m{:03d}/output/mix_emcee/run01/emcee_chain.npy".format(m))
        new_df.set_value(ind, 'inference', True)
    except:
        new_df.set_value(ind, 'inference', False)

## $H-$band

In [11]:
ms = np.hstack((np.arange(100, 111), np.arange(112,123+1)))

In [14]:
for m in ms:
    ind = new_df.index[new_df.m_val == m]
    try:
        ws = np.load("../sf/m{:03d}/output/mix_emcee/run02/emcee_chain.npy".format(m))
        new_df.set_value(ind, 'inference', True)
    except:
        new_df.set_value(ind, 'inference', False)

The orders that did not successfully complete Starfish runs.

In [17]:
new_df.inference[new_df.inference != new_df.inference] = False

//anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


## Prepare the column names.

In [18]:
cs = new_df.columns[7:-5:3]
bases = cs.str[:-4]
base_inds = np.hstack([np.arange(6), np.arange(8, 14), [6, 7]])
suffixes = ['_05p', '_50p', '_95p']
base_dict = {key:val for key, val in zip(bases, base_inds)}
suffix_dict = {key:val for key, val in zip(suffixes, [5, 50, 95])}

We want the last 200 walkers.

In [30]:
n_s = 200

## $K-$band

In [27]:
ms = np.arange(72, 94+1)

In [31]:
for m in ms:
    ind = new_df.index[new_df.m_val == m]
    ws = np.load("../sf/m{:03d}/output/mix_emcee/run01/emcee_chain.npy".format(m))
    burned = ws[:, -1*n_s:,:]
    xs, ys, zs = burned.shape
    fc = burned.reshape(xs*ys, zs)
    bb = 10**fc[:,7] / ( 10**fc[:,5] + 10**fc[:,7] )
    for suffix in suffixes:
        new_df.set_value(ind, 'ff'+suffix, np.percentile(bb, suffix_dict[suffix]))
        for base in bases:
            new_df.set_value(ind, base+suffix, np.percentile(fc[:, base_dict[base]], suffix_dict[suffix]))

## $H-$band

In [32]:
ms = np.hstack((np.arange(100, 111), np.arange(112,123+1)))

In [33]:
for m in ms:
    ind = new_df.index[new_df.m_val == m]
    ws = np.load("../sf/m{:03d}/output/mix_emcee/run02/emcee_chain.npy".format(m))
    burned = ws[:, -1*n_s:,:]
    xs, ys, zs = burned.shape
    fc = burned.reshape(xs*ys, zs)
    bb = 10**fc[:,7] / ( 10**fc[:,5] + 10**fc[:,7] )
    for suffix in suffixes:
        new_df.set_value(ind, 'ff'+suffix, np.percentile(bb, suffix_dict[suffix]))
        for base in bases:
            new_df.set_value(ind, base+suffix, np.percentile(fc[:, base_dict[base]], suffix_dict[suffix]))

In [35]:
new_df.head()

,wl_start,wl_end,band,number,tell_trans,m_val,wl_center,Teff_05p,Teff_50p,Teff_95p,...,Teff2_05p,Teff2_50p,Teff2_95p,logO2_05p,logO2_50p,logO2_95p,ff_05p,ff_50p,ff_95p,inference
0,18092.0,18346.0,H,0,0.117487,98,18219.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,17914.0,18165.0,H,1,0.342583,99,18039.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,17739.0,17988.0,H,2,0.666035,100,17863.5,4119.604630,4312.672222,4474.515407,...,2703.707416,2765.367136,2900.392667,-0.058854,0.051357,0.139004,0.734685,0.794156,0.832494,True
3,17568.0,17815.0,H,3,0.827089,101,17691.5,3625.364642,4199.583018,4463.705141,...,2757.251605,2916.251194,3075.061624,-0.382951,-0.004874,0.110811,0.566891,0.792576,0.844624,True
4,17400.0,17645.0,H,4,0.904758,102,17522.5,4176.946167,4336.687926,4473.679807,...,2702.575311,2751.474163,2864.956469,0.029823,0.117106,0.199920,0.772381,0.819419,0.858518,True


In [36]:
new_df.to_csv('../data/analysis/IGRINS_mix_K_Run01_H_Run02.csv', index=False)

The end.